In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt
import re

In [6]:
# Document 1 Cleaning 
def clean_func(path, sheet):
    '''
    Fucntion that creates one spreadsheet for each year's media spending data
    - path: where you save the file
    - sheet: sheet number([1 ,2, 3, 4, 5]) corresponds to year [2020, 2019, 2018, 2017, 2016]
    '''
    year_mapping = {1:2020, 2:2019, 3:2018, 4:2017, 5:2016}
    year = year_mapping[sheet]
    data = pd.read_excel(path, sheet_name = sheet, 
                          header = 6, skipfooter = 1, usecols = 'A:BB')
    data.rename(columns = {'Unnamed: 0' : 'Media', 'Unnamed: 1' : 'Tactic'}, inplace = True)
    data.dropna(how="all", inplace=True)
    data = data.fillna(0)
    swap = pd.melt(data, id_vars = ['Media', 'Tactic'], var_name = 'Week', value_name = "Spending")
    week_num = pd.read_excel(path, sheet_name = sheet, 
                      header = 5, nrows = 2, usecols = 'A:BB')
    week_num.drop(columns = ['Unnamed: 0', 'Unnamed: 1'], inplace = True)
    week_start = []
    week_start_dict = {}
    for j in week_num.columns:
        pd.to_datetime(j, format = '%Y-%m-%d')
        week_start.append(j)
    for k in range(1,53):
        week_start_dict[k] = week_start[k-1]
    swap['Week_start'] = swap['Week'].map(week_start_dict)
    mapping1 = {}
    for i in range(1, 53):
        if i <= 13:
            mapping1[i] = 1
        elif i <= 26:
            mapping1[i] = 2
        elif i <= 39:
            mapping1[i] = 3
        else:
            mapping1[i] =4
    swap['Year'] = year
    swap['Quarter'] = swap['Week'].map(mapping1)
    month_list = []
    for date in swap['Week_start']:
        if date.year != year:
            month_list.append(1)
        else:
            month_list.append(date.month)
    swap['Month'] = month_list
    swap = swap[['Media', 'Tactic', 'Week_start', 'Year', 'Quarter', 'Month', 'Week', 'Spending']]
    return swap

check20 = clean_func('DATA/KH_marketing_data_cat.xlsx', 1)
check19 = clean_func('DATA/KH_marketing_data_cat.xlsx', 2)
check18 = clean_func('DATA/KH_marketing_data_cat.xlsx', 3)
check17 = clean_func('DATA/KH_marketing_data_cat.xlsx', 4)
check16 = clean_func('DATA/KH_marketing_data_cat.xlsx', 5)

concat_data = pd.concat([check20, check19, check18, check17, check16], axis = 0, ignore_index = True)
concat_data.to_csv('KH_Media_Spend_Tactic_Cleaned.csv', index = False)

In [7]:
# Document 2 Data Cleaning 

mapping = pd.read_excel("DATA/2. 2016-2020 Consumption by week_USC CONFIDENTIAL.xlsx", sheet_name = 0)

# clean up mapping data
mapping = mapping.drop(["CONFIDENTIAL Contains Proprietary Information","Unnamed: 2"], axis =  1) # drop first column
mapping = mapping.rename(columns = {"Unnamed: 1":"index","Unnamed: 3":"region"}) # rename columns
mapping = mapping.drop(0, axis =  0) # drop first row
mapping["index"] = mapping["index"].astype(int)

master = pd.DataFrame()

# read in different tabs data
for i in range(1,66,1):
    
    # read in different tab
    consump = pd.read_excel("DATA/2. 2016-2020 Consumption by week_USC CONFIDENTIAL.xlsx", sheet_name = i)
    consump = consump.rename(columns = consump.iloc[7,:]) # use row 7th for column name
    consump = consump.drop([0,1,2,3,4,5,6,7], axis = 0) # delete extra row
    consump["index"] = i
    master = pd.concat([consump, master], ignore_index=True)

master = master.rename(columns = {np.NaN:"product_name"})
master_melt = pd.melt(master, id_vars= ["product_name", "index"], var_name = "week_ending", value_name= "consumption")
master_melt["week_ending"] = master_melt["week_ending"].apply(lambda x: pd.to_datetime(x.replace("Week Ending ","")))

master_melt["year"] = master_melt["week_ending"].dt.year
master_melt["quarter"] = master_melt["week_ending"].dt.quarter
master_melt["month"] = master_melt["week_ending"].dt.month
master_melt["week"] = master_melt["week_ending"].dt.week

master_merged_data = pd.merge(master_melt, mapping, on = "index", how = "left")
master_merged_data = master_merged_data.drop("index", axis = 1)
master_merged_data = master_merged_data.rename(columns = {"product_name":"Product", "week_ending":"Date"})
master_merged_data["region"] = master_merged_data["region"].apply(lambda x: x.replace(" - Multi Outlet",""))

# Average Price Data
avg_price = pd.read_excel("DATA/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx", sheet_name=1)
avg_price_master = pd.DataFrame()

# read in different tabs data
for i in range(1,66,1):
    
    # read in different tab
    avg_price = pd.read_excel("DATA/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx", sheet_name = i)
    avg_price = avg_price.rename(columns = avg_price.iloc[7,:]) # use row 7th for column name
    avg_price = avg_price.drop([0,1,2,3,4,5,6,7], axis = 0) # delete extra row
    avg_price["index"] = i
    avg_price_master = pd.concat([avg_price, avg_price_master], ignore_index=True)

# transform
avg_price_master = avg_price_master.rename(columns = {np.NaN:"Product"})
avg_price_master = pd.melt(avg_price_master, id_vars= ["Product", "index"], var_name = "Date", value_name= "avg_price")

# clean up data
avg_price_master["Date"] = avg_price_master["Date"].apply(lambda x: pd.to_datetime(x.replace("Week Ending ","")))

avg_price_master["year"] = avg_price_master["Date"].dt.year
avg_price_master["quarter"] = avg_price_master["Date"].dt.quarter
avg_price_master["month"] = avg_price_master["Date"].dt.month
avg_price_master["week"] = avg_price_master["Date"].dt.week

avg_price_master = pd.merge(avg_price_master, mapping, on = "index", how = "left")
avg_price_master["region"] = avg_price_master["region"].apply(lambda x: x.replace(" - Multi Outlet",""))

# ACV Data 

# initialize a master dataframe
acv_master = pd.DataFrame()

# read in different tabs data
for i in range(1,66,1):
    
    # read in different tab
    acv = pd.read_excel("DATA/5.1. 2016-2020 ACV by week_USC_UPDATED CONFIDENTIAL.xlsx", sheet_name = i)
    acv = acv.rename(columns = acv.iloc[7,:]) # use row 7th for column name
    acv = acv.drop([0,1,2,3,4,5,6,7], axis = 0) # delete extra row
    acv["index"] = i
    acv_master = pd.concat([acv, acv_master], ignore_index=True)

acv_master = acv_master.rename(columns = {np.NaN:"Product"})
acv_master = pd.melt(acv_master, id_vars= ["Product", "index"], var_name = "Date", value_name= "ACV")

acv_master["Date"] = acv_master["Date"].apply(lambda x: pd.to_datetime(x.replace("Week Ending ","")))
acv_master["year"] = acv_master["Date"].dt.year
acv_master["quarter"] = acv_master["Date"].dt.quarter
acv_master["month"] = acv_master["Date"].dt.month
acv_master["week"] = acv_master["Date"].dt.week

acv_master = pd.merge(acv_master, mapping, on = "index", how = "left")
acv_master["region"] = acv_master["region"].apply(lambda x: x.replace(" - Multi Outlet",""))

consumption_data = master_merged_data.rename(columns={"consumption":"value"})
consumption_data["value_type"] = "consumption"

average_price_data = avg_price_master.rename(columns={"avg_price":"value"})
average_price_data = average_price_data.drop("index", axis = 1)
average_price_data["value_type"] = "average_price"

acv_data = acv_master.rename(columns={"ACV":"value"})
acv_data = acv_data.drop("index", axis = 1)
acv_data["value_type"] = "acv"

# combine all three
master_data = pd.concat([consumption_data,average_price_data,acv_data])

unmelton_data = master_data.pivot_table(index=["Product","Date","year","quarter","month","week","region"],columns="value_type", values="value",aggfunc="sum")

In [2]:
# Document 3 Data Cleaning

# Reads Data, renmaes columns
df=pd.read_excel('DATA/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx',header=8,sheet_name=5)

# Renames columns
df.rename(columns={'Unnamed: 0':'Product'},inplace=True)
new_columns=pd.date_range(start = '2015-01-11', end = '2020-09-13',freq='7D')
for i in range(1,298):
    df.rename(columns={df.columns[i]:new_columns[i-1]},inplace=True)
df.rename(columns={'Week Ending 01-11-15':new_columns[0]})

# Adjusts Dataframe structure
melt_data = pd.melt(df,id_vars=['Product'], var_name = 'Date', value_name = 'AVG')

# Add date time information
melt_data['Year']=melt_data['Date'].dt.year
melt_data['Quarter']=melt_data['Date'].dt.quarter
melt_data['Month']=melt_data['Date'].dt.month
melt_data['Week']=melt_data['Date'].dt.week

# Gets geography information
geo_info = pd.read_excel('DATA/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx',sheet_name='Index',header=1)
geo_info=geo_info.drop(['Unnamed: 0','Index'],axis=1)
geo=geo_info['Geography']
geo_clean=pd.Series()
geo_city_and_state=pd.Series()

# Loops through each of the locations
for i in geo:
    geo_clean[i]=i.split('-')[0]
for i in geo_clean:
    geo_city_and_state[i]=i.split(',')
len(geo_city_and_state)

for i in geo_city_and_state:
    if len(i)<2:
        geo_city_and_state[i]=geo_city_and_state[i]+geo_city_and_state[i]
geo_city_and_state[37]=['New Orleans,LA/Mobile','AL']
geo_city_and_state[20]=['Hartford,CT/Springfield','MA']

avg_us=melt_data.copy()
melt_avg=avg_us.copy()
for i in range(2,66):
    df=pd.read_excel('DATA/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx',sheet_name=i,header=8)
    df.rename(columns={'Unnamed: 0':'Product'},inplace=True)
    new_columns=pd.date_range(start = '2015-01-11', end = '2020-09-13',freq='7D')
    for j in range(1,298):
        df.rename(columns={df.columns[j]:new_columns[j-1]},inplace=True)
    melt_data = pd.melt(df,id_vars=['Product'], var_name = 'Date', value_name = 'AVG')
    melt_data['Year']=melt_data['Date'].dt.year
    melt_data['Quarter']=melt_data['Date'].dt.quarter
    melt_data['Month']=melt_data['Date'].dt.month
    melt_data['Week']=melt_data['Date'].dt.week
    melt_data['City']=geo_city_and_state[i-1][0]
    melt_data['State']=geo_city_and_state[i-1][1]
    summary=melt_data.groupby(['Product','Year']).agg({'AVG':'mean'})
    melt_avg=pd.concat([melt_avg,melt_data],ignore_index = True, axis = 0)
    
#melt_avg.to_csv('cleaned_avg_price')

/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [3]:
# Document 4 Cleaning

geo = pd.read_excel('DATA/4. 2016-2020 base price_, feature price_, promo price by week by SKU USC CONFIDENTIAL.xlsx',sheet_name = 'Index')
geo_dict = {}
for i in range(1,66):
    geo_dict[i] = geo.iloc[i,3]
    
temp= []
for i in range(1,66):
    temp.append(re.sub(r' - Multi Outlet','',geo_dict[i]))

## split to city and state
geo_city = []
geo_state = []
for i in temp:
    splitcity = re.split(r', ', i)
    if len(splitcity) == 2:
        geo_city.append(splitcity[0])
        geo_state.append(splitcity[1])
    else: 
        geo_city.append(splitcity[0])
        geo_state.append(splitcity[0])

## create new geo dict
geo_dict_new = {i: [geo_city[i-1],geo_state[i-1]] for i in range(1,66)}

for i in range(1,66):
    # Read data
    tmp = pd.read_excel('DATA/4. 2016-2020 base price_, feature price_, promo price by week by SKU USC CONFIDENTIAL.xlsx',sheet_name = i)
    
    # Initial data cleaning
    tmp.drop(tmp.index[0:7],inplace=True)
    tmp.rename(columns = {'Unnamed: 0':'Product'},inplace=True)
    for j in range(1,tmp.shape[1]):
        tmp.rename(columns={f'Unnamed: {j}':f'{tmp.iloc[0,j]}'}, inplace = True)
    tmp = tmp[1:]
    
    p1 = tmp.iloc[:,0:298]
    p1_melt = pd.melt(p1,id_vars='Product',var_name= 'Date',value_name = 'Weighted Average Base Price Per Unit')
    p2= tmp.iloc[:,298:595]
    p2['Product'] =tmp['Product']
    p2_melt = pd.melt(p2,id_vars='Product',var_name= 'Date',value_name = 'Weighted Average Base Price Per Unit Any Price Reduction')
    p3= tmp.iloc[:,595:892]
    p3['Product'] = tmp['Product']
    p3_melt = pd.melt(p3,id_vars='Product',var_name= 'Date',value_name = 'Weighted Average Base Price Per Unit Any Feature')
    p4= tmp.iloc[:,892:1189]
    p4['Product'] = tmp['Product']
    p4_melt = pd.melt(p4,id_vars='Product',var_name= 'Date',value_name = 'Weighted Average Base Price Per Unit Any Display')
    
    melt_tmp = p1_melt
    melt_tmp['Weighted Average Base Price Per Unit Any Price Reduction'] = p2_melt['Weighted Average Base Price Per Unit Any Price Reduction']
    melt_tmp['Weighted Average Base Price Per Unit Any Feature']  = p3_melt['Weighted Average Base Price Per Unit Any Feature']
    melt_tmp['Weighted Average Base Price Per Unit Any Display'] = p4_melt['Weighted Average Base Price Per Unit Any Display']
    
    melt_tmp['Date'] = melt_tmp['Date'].str[12:]
    melt_tmp['Date'] = pd.to_datetime(melt_tmp['Date'])
    
    
    # Create relevant columns
    melt_tmp['Year'] = melt_tmp['Date'].dt.year
    melt_tmp['Quarter'] = melt_tmp['Date'].dt.quarter
    melt_tmp['Month'] = melt_tmp['Date'].dt.month
    melt_tmp['Week'] = melt_tmp['Date'].dt.week
    melt_tmp['City'] = geo_dict_new[i][0]
    melt_tmp['State'] = geo_dict_new[i][1]
    
    # Concatenate different sheets
    if i==1:
        melt_data = melt_tmp
    else:
        melt_data = pd.concat([melt_data, melt_tmp], ignore_index = True, axis = 0)

#melt_data.to_csv('4.Cleaned_Data_Oct26_separate_prices.csv', index=False) 

/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [4]:
# Document 5 Cleaning
# 1. Create a geographic dict
geo_info = pd.read_excel('DATA/5. 2016-2020 ACV by Display Feature by week_USC CONFIDENTIAL.xlsx',sheet_name = 'Index')
geo_dict = {}
for i in range(1,66): 
    geo_dict[i] = geo_info.iloc[i,3]

# 2. ACV data cleaning and merging
for i in range(1,66):
    # Read data
    tmp = pd.read_excel('DATA/5. 2016-2020 ACV by Display Feature by week_USC CONFIDENTIAL.xlsx',sheet_name = i)
    
    # Initial data cleaning
    tmp.drop(tmp.index[0:7],inplace=True)
    tmp.rename(columns = {'Unnamed: 0':'Product'},inplace=True)
    for j in range(1,298):
        tmp.rename(columns={f'Unnamed: {j}':f'{tmp.iloc[0,j]}'}, inplace = True)
    tmp = tmp[1:]
    melt_tmp = pd.melt(tmp,id_vars=['Product'], var_name = 'Date', value_name = 'ACV')
    melt_tmp['Date'] = melt_tmp['Date'].str[12:]
    melt_tmp['Date'] = pd.to_datetime(melt_tmp['Date'])
    
    # Create relevant columns
    melt_tmp['Year'] = melt_tmp['Date'].dt.year
    melt_tmp['Month'] = melt_tmp['Date'].dt.month
    melt_tmp['Week'] = melt_tmp['Date'].dt.week
    melt_tmp['Quarter'] = melt_tmp['Date'].dt.quarter
    melt_tmp['Geo'] = geo_dict[i]
    
    list_state_equal_to_city = [1,35,37,57,64]
    list_special = [21,38] # New Orleans, LA/Mobile, AL - Multi Outlet////Hartford, CT/Springfield, MA - Multi Outlet
    
    if i in list_state_equal_to_city: 
        melt_tmp['City'],_ = melt_tmp['Geo'].str.split("-").str
        melt_tmp['State'] = melt_tmp['City']
    elif i in list_special: 
        city1,state_city,state2 = melt_tmp['Geo'].str.split(",").str
        state1, city2 = state_city.str.split("/").str
        state2, _ = state2.str.split("-").str
        melt_tmp['City'] = city1+'/'+city2
        melt_tmp['State'] = state1+'/'+state2
    else: 
        melt_tmp['City'], melt_tmp['State']= melt_tmp['Geo'].str.split(",").str
        melt_tmp['State'],_ = melt_tmp['State'].str.split("-").str
    
    # Concatenate ACV of different sheets
    if i==1:
        melt_acv = melt_tmp
    else:
        melt_acv = pd.concat([melt_acv, melt_tmp], ignore_index = True, axis = 0)
    
    # test 
    
# Drop the column 'Geo' 
melt_acv.drop(columns = 'Geo',inplace = True)

/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Columnar iteration over characters w

In [5]:
# Document 14 Cleaning
data = pd.read_excel('DATA/14. Ongoing promos and placement by retailer USC CONFIDENTIAL.xlsx')
updated_data = data.copy()

columns = updated_data.iloc[:,6:11].columns 
for col in columns:
    updated_data[col] = updated_data[col].map({'Yes': 1,'Yes ': 1, 'No': 0})
    
updated_data['Holiday'] = updated_data['Holiday'].map({'Christmas': 'Christmas/NY',
                                                       'Christmas/NY': 'Christmas/NY',
                                                       'Memorial Week ': 'Memorial Week',
                                                       'Thanskgiving ': 'Thanksgiving',
                                                       'Easter Week ': 'Easter Week',
                                                       'July 4th ': 'July 4th',
                                                       'Halloween ': 'Halloween',
                                                       'SuperBowl ': 'Super Bowl',
                                                       'Labor Day ': 'Labor Day',
                                                       'Easter Weeks': 'Easter Week'})

updated_data['Holiday'].fillna('None', inplace = True)
updated_data.fillna(0, inplace = True)

updated_data.columns = ['store', 'product', 'platform', 'week_start', 'week_end', 'holiday', 'chicken_meal_deal', 
                        'sec_deli', 'sec_bakery', 'sec_center', 'retail_cover', 'volume_sales', 'volume_sales_merch',
                        'volume_sales_price_reduce', 'volume_sales_feature', 'volume_sales_display', 'dollar_sales',
                        'dollar_sales_merch', 'dollar_sales_price_reduce', 'dollar_sales_feature', 'dollar_sales_display',
                        'unit_sales', 'unit_sales_merch', 'unit_sales_price_reduce', 'unit_sales_feature', 'unit_sales_display',
                        'volume_perMM_ACV', 'volume_perMM_ACV_merch', 'volume_perMM_ACV_feature', 'volume_perMM_ACV_display',
                        'volume_per_store_selling', 'volume_per_store_selling_merch', 'volume_per_store_selling_price_reduce',
                        'volume_per_store_selling_feature', 'volume_per_store_selling_display', 'price_per_unit', 'price_per_unit_merch',
                        'price_per_unit_price_reduce', 'price_per_unit_feature', 'price_per_unit_display', 'avg_week_ACV_distribution',
                        'avg_week_ACV_distribution_merch', 'avg_week_ACV_distribution_price_reduce', 'avg_week_ACV_distribution_feature',
                        'avg_week_ACV_distribution_display']

updated_data['year'] = updated_data['week_end'].dt.year
updated_data['month'] = updated_data['week_end'].dt.month
updated_data['week'] = updated_data['week_end'].dt.week
updated_data['quarter'] = updated_data['week_end'].dt.quarter

updated_data.loc[(updated_data['week'] == 1) | ((updated_data['week'] <= 52) & (updated_data['week'] >= 50)), 'holiday'] = 'Christmas/NY'
updated_data.loc[(updated_data['week'] <= 16) & (updated_data['week'] >= 13), 'holiday'] = 'Easter Week'
updated_data.loc[(updated_data['week'] <= 45) & (updated_data['week'] >= 43), 'holiday'] = 'Halloween'
updated_data.loc[(updated_data['week'] <= 27) & (updated_data['week'] >= 26), 'holiday'] = 'July 4th'
updated_data.loc[updated_data['week'] == 36, 'holiday'] = 'Labor Day'
updated_data.loc[(updated_data['week'] <= 21) & (updated_data['week'] >= 20), 'holiday'] = 'Memorial Week'
updated_data.loc[(updated_data['week'] <= 48) & (updated_data['week'] >= 46), 'holiday'] = 'Thanksgiving'
updated_data.loc[updated_data['holiday'] == 'None', 'holiday_binary'] = 0
updated_data['holiday_binary'].fillna(1, inplace = True)

# updated_data.to_csv('data/14SalesByRetailLocation.csv')

In [ ]:
# Document 14 Transformation for Analysis

data = pd.read_csv('DATA/14SalesByRetailLocation.csv', parse_dates = ['week_start', 'week_end'], index_col = 0)

# Finds Volume Aggregation by Product by Week
volume = data.groupby(['product','week_start']).agg({'volume_sales': 'sum',
                                                     'volume_sales_merch': 'sum',
                                                     'volume_sales_price_reduce':'sum',
                                                     'volume_sales_feature': 'sum',
                                                     'volume_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
volume['year'] = volume['week_start'].dt.year
volume['month'] = volume['week_start'].dt.month
volume['week'] = volume['week_start'].dt.week
volume['quarter'] = volume['week_start'].dt.quarter
volume.loc[(volume['week'] == 1) | ((volume['week'] <= 52) & (volume['week'] >= 50)), 'holiday'] = 'Christmas/NY'
volume.loc[(volume['week'] <= 16) & (volume['week'] >= 13), 'holiday'] = 'Easter Week'
volume.loc[(volume['week'] <= 45) & (volume['week'] >= 43), 'holiday'] = 'Halloween'
volume.loc[(volume['week'] <= 27) & (volume['week'] >= 26), 'holiday'] = 'July 4th'
volume.loc[volume['week'] == 36, 'holiday'] = 'Labor Day'
volume.loc[(volume['week'] <= 21) & (volume['week'] >= 20), 'holiday'] = 'Memorial Week'
volume.loc[(volume['week'] <= 48) & (volume['week'] >= 46), 'holiday'] = 'Thanksgiving'
volume['holiday'].fillna('None', inplace = True)
volume.loc[volume['holiday'] == 'None', 'holiday_binary'] = 0
volume['holiday_binary'].fillna(1, inplace = True)
# volume.to_csv('data/14VolumeByProduct.csv')

# Finds Dollar Sales Aggregations by Product by Week
dollar = data.groupby(['product','week_start']).agg({'dollar_sales': 'sum',
                                                     'dollar_sales_merch': 'sum',
                                                     'dollar_sales_price_reduce':'sum',
                                                     'dollar_sales_feature': 'sum',
                                                     'dollar_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
dollar['year'] = dollar['week_start'].dt.year
dollar['month'] = dollar['week_start'].dt.month
dollar['week'] = dollar['week_start'].dt.week
dollar['quarter'] = dollar['week_start'].dt.quarter
dollar.loc[(dollar['week'] == 1) | ((dollar['week'] <= 52) & (dollar['week'] >= 50)), 'holiday'] = 'Christmas/NY'
dollar.loc[(dollar['week'] <= 16) & (dollar['week'] >= 13), 'holiday'] = 'Easter Week'
dollar.loc[(dollar['week'] <= 45) & (dollar['week'] >= 43), 'holiday'] = 'Halloween'
dollar.loc[(dollar['week'] <= 27) & (dollar['week'] >= 26), 'holiday'] = 'July 4th'
dollar.loc[dollar['week'] == 36, 'holiday'] = 'Labor Day'
dollar.loc[(dollar['week'] <= 21) & (dollar['week'] >= 20), 'holiday'] = 'Memorial Week'
dollar.loc[(dollar['week'] <= 48) & (dollar['week'] >= 46), 'holiday'] = 'Thanksgiving'
dollar['holiday'].fillna('None', inplace = True)
dollar.loc[dollar['holiday'] == 'None', 'holiday_binary'] = 0
dollar['holiday_binary'].fillna(1, inplace = True)
#dollar.to_csv('data/14DollarByProduct.csv')

# Finds Unit Aggregations by Product by Week
unit = data.groupby(['product','week_start']).agg({'unit_sales': 'sum',
                                                     'unit_sales_merch': 'sum',
                                                     'unit_sales_price_reduce':'sum',
                                                     'unit_sales_feature': 'sum',
                                                     'unit_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
unit['year'] = unit['week_start'].dt.year
unit['month'] = unit['week_start'].dt.month
unit['week'] = unit['week_start'].dt.week
unit['quarter'] = unit['week_start'].dt.quarter
unit.loc[(unit['week'] == 1) | ((unit['week'] <= 52) & (unit['week'] >= 50)), 'holiday'] = 'Christmas/NY'
unit.loc[(unit['week'] <= 16) & (unit['week'] >= 13), 'holiday'] = 'Easter Week'
unit.loc[(unit['week'] <= 45) & (unit['week'] >= 43), 'holiday'] = 'Halloween'
unit.loc[(unit['week'] <= 27) & (unit['week'] >= 26), 'holiday'] = 'July 4th'
unit.loc[unit['week'] == 36, 'holiday'] = 'Labor Day'
unit.loc[(unit['week'] <= 21) & (unit['week'] >= 20), 'holiday'] = 'Memorial Week'
unit.loc[(unit['week'] <= 48) & (unit['week'] >= 46), 'holiday'] = 'Thanksgiving'
unit['holiday'].fillna('None', inplace = True)
unit.loc[unit['holiday'] == 'None', 'holiday_binary'] = 0
unit['holiday_binary'].fillna(1, inplace = True)
#unit.to_csv('data/14UnitByProduct.csv')

# Finds Price Aggregations by Product by Week
price = data.groupby(['product','week_start']).agg({'price_per_unit': 'mean',
                                                     'price_per_unit_merch': 'mean',
                                                     'price_per_unit_price_reduce':'mean',
                                                     'price_per_unit_feature': 'mean',
                                                     'price_per_unit_display':'mean',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
price['year'] = price['week_start'].dt.year
price['month'] = price['week_start'].dt.month
price['week'] = price['week_start'].dt.week
price['quarter'] = price['week_start'].dt.quarter
price.loc[(price['week'] == 1) | ((price['week'] <= 52) & (price['week'] >= 50)), 'holiday'] = 'Christmas/NY'
price.loc[(price['week'] <= 16) & (price['week'] >= 13), 'holiday'] = 'Easter Week'
price.loc[(price['week'] <= 45) & (price['week'] >= 43), 'holiday'] = 'Halloween'
price.loc[(price['week'] <= 27) & (price['week'] >= 26), 'holiday'] = 'July 4th'
price.loc[price['week'] == 36, 'holiday'] = 'Labor Day'
price.loc[(price['week'] <= 21) & (price['week'] >= 20), 'holiday'] = 'Memorial Week'
price.loc[(price['week'] <= 48) & (price['week'] >= 46), 'holiday'] = 'Thanksgiving'
price['holiday'].fillna('None', inplace = True)
price.loc[price['holiday'] == 'None', 'holiday_binary'] = 0
price['holiday_binary'].fillna(1, inplace = True)
#price.to_csv('data/14PriceByProduct.csv')

# Finds Volume Aggregations by Week
volume = data.groupby(['week_start']).agg({'volume_sales': 'sum',
                                                     'volume_sales_merch': 'sum',
                                                     'volume_sales_price_reduce':'sum',
                                                     'volume_sales_feature': 'sum',
                                                     'volume_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
volume['year'] = volume['week_start'].dt.year
volume['month'] = volume['week_start'].dt.month
volume['week'] = volume['week_start'].dt.week
volume['quarter'] = volume['week_start'].dt.quarter
volume.loc[(volume['week'] == 1) | ((volume['week'] <= 52) & (volume['week'] >= 50)), 'holiday'] = 'Christmas/NY'
volume.loc[(volume['week'] <= 16) & (volume['week'] >= 13), 'holiday'] = 'Easter Week'
volume.loc[(volume['week'] <= 45) & (volume['week'] >= 43), 'holiday'] = 'Halloween'
volume.loc[(volume['week'] <= 27) & (volume['week'] >= 26), 'holiday'] = 'July 4th'
volume.loc[volume['week'] == 36, 'holiday'] = 'Labor Day'
volume.loc[(volume['week'] <= 21) & (volume['week'] >= 20), 'holiday'] = 'Memorial Week'
volume.loc[(volume['week'] <= 48) & (volume['week'] >= 46), 'holiday'] = 'Thanksgiving'
volume['holiday'].fillna('None', inplace = True)
volume.loc[volume['holiday'] == 'None', 'holiday_binary'] = 0
volume['holiday_binary'].fillna(1, inplace = True)
#volume.to_csv('data/14VolumeByWeek.csv')

# Finds Dollar Aggregations by Week
dollar = data.groupby(['product','week_start']).agg({'dollar_sales': 'sum',
                                                     'dollar_sales_merch': 'sum',
                                                     'dollar_sales_price_reduce':'sum',
                                                     'dollar_sales_feature': 'sum',
                                                     'dollar_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
dollar['year'] = dollar['week_start'].dt.year
dollar['month'] = dollar['week_start'].dt.month
dollar['week'] = dollar['week_start'].dt.week
dollar['quarter'] = dollar['week_start'].dt.quarter
dollar.loc[(dollar['week'] == 1) | ((dollar['week'] <= 52) & (dollar['week'] >= 50)), 'holiday'] = 'Christmas/NY'
dollar.loc[(dollar['week'] <= 16) & (dollar['week'] >= 13), 'holiday'] = 'Easter Week'
dollar.loc[(dollar['week'] <= 45) & (dollar['week'] >= 43), 'holiday'] = 'Halloween'
dollar.loc[(dollar['week'] <= 27) & (dollar['week'] >= 26), 'holiday'] = 'July 4th'
dollar.loc[dollar['week'] == 36, 'holiday'] = 'Labor Day'
dollar.loc[(dollar['week'] <= 21) & (dollar['week'] >= 20), 'holiday'] = 'Memorial Week'
dollar.loc[(dollar['week'] <= 48) & (dollar['week'] >= 46), 'holiday'] = 'Thanksgiving'
dollar['holiday'].fillna('None', inplace = True)
dollar.loc[dollar['holiday'] == 'None', 'holiday_binary'] = 0
dollar['holiday_binary'].fillna(1, inplace = True)
#dollar.to_csv('data/14DollarByWeek.csv')

# Finds Unit Sales Aggregations by Week
unit = data.groupby(['product','week_start']).agg({'unit_sales': 'sum',
                                                     'unit_sales_merch': 'sum',
                                                     'unit_sales_price_reduce':'sum',
                                                     'unit_sales_feature': 'sum',
                                                     'unit_sales_display':'sum',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
unit['year'] = unit['week_start'].dt.year
unit['month'] = unit['week_start'].dt.month
unit['week'] = unit['week_start'].dt.week
unit['quarter'] = unit['week_start'].dt.quarter
unit.loc[(unit['week'] == 1) | ((unit['week'] <= 52) & (unit['week'] >= 50)), 'holiday'] = 'Christmas/NY'
unit.loc[(unit['week'] <= 16) & (unit['week'] >= 13), 'holiday'] = 'Easter Week'
unit.loc[(unit['week'] <= 45) & (unit['week'] >= 43), 'holiday'] = 'Halloween'
unit.loc[(unit['week'] <= 27) & (unit['week'] >= 26), 'holiday'] = 'July 4th'
unit.loc[unit['week'] == 36, 'holiday'] = 'Labor Day'
unit.loc[(unit['week'] <= 21) & (unit['week'] >= 20), 'holiday'] = 'Memorial Week'
unit.loc[(unit['week'] <= 48) & (unit['week'] >= 46), 'holiday'] = 'Thanksgiving'
unit['holiday'].fillna('None', inplace = True)
unit.loc[unit['holiday'] == 'None', 'holiday_binary'] = 0
unit['holiday_binary'].fillna(1, inplace = True)
#unit.to_csv('data/14UnitByWeek.csv')

# Finds Price Aggregations by Week
price = data.groupby(['product','week_start']).agg({'price_per_unit': 'mean',
                                                     'price_per_unit_merch': 'mean',
                                                     'price_per_unit_price_reduce':'mean',
                                                     'price_per_unit_feature': 'mean',
                                                     'price_per_unit_display': 'mean',
                                                     'chicken_meal_deal':'mean',
                                                     'sec_deli':'mean',
                                                     'sec_bakery':'mean',
                                                     'sec_center':'mean',
                                                     'retail_cover':'mean'}).reset_index()
price['year'] = price['week_start'].dt.year
price['month'] = price['week_start'].dt.month
price['week'] = price['week_start'].dt.week
price['quarter'] = price['week_start'].dt.quarter
price.loc[(price['week'] == 1) | ((price['week'] <= 52) & (price['week'] >= 50)), 'holiday'] = 'Christmas/NY'
price.loc[(price['week'] <= 16) & (price['week'] >= 13), 'holiday'] = 'Easter Week'
price.loc[(price['week'] <= 45) & (price['week'] >= 43), 'holiday'] = 'Halloween'
price.loc[(price['week'] <= 27) & (price['week'] >= 26), 'holiday'] = 'July 4th'
price.loc[price['week'] == 36, 'holiday'] = 'Labor Day'
price.loc[(price['week'] <= 21) & (price['week'] >= 20), 'holiday'] = 'Memorial Week'
price.loc[(price['week'] <= 48) & (price['week'] >= 46), 'holiday'] = 'Thanksgiving'
price['holiday'].fillna('None', inplace = True)
price.loc[price['holiday'] == 'None', 'holiday_binary'] = 0
price['holiday_binary'].fillna(1, inplace = True)
#price.to_csv('data/14PriceByWeek.csv')

volume_byproduct = pd.read_csv('DATA/14VolumeByProduct.csv', index_col = 0)
dollar_byproduct = pd.read_csv('DATA/14DollarByProduct.csv', index_col = 0)
unit_byproduct = pd.read_csv('DATA/14UnitByProduct.csv', index_col = 0)
price_byproduct = pd.read_csv('DATA/14PriceByProduct.csv', index_col = 0)
volume_byweek = pd.read_csv('DATA/14VolumeByWeek.csv', index_col = 0)
dollar_byweek = pd.read_csv('DATA/14DollarByWeek.csv', index_col = 0)
unit_byweek = pd.read_csv('DATA/14UnitByWeek.csv', index_col = 0)
price_byweek = pd.read_csv('DATA/14PriceByWeek.csv', index_col = 0)
spending = pd.read_csv('DATA/1. KH_Media_Spend_Tactic_Cleaned.csv')

# Adds Media Spending Data 
def addMedia(data, media, year, filename): 
    subset = media.loc[media['Year'] == year]
    data = data.loc[data['year'] == year]
    tactics = subset.groupby(['Week', 'Tactic']).agg({'Spending':'sum'}).reset_index()
    data = pd.concat([data, pd.DataFrame(columns = list(subset.Media.unique()))])
    data = pd.concat([data, pd.DataFrame(columns = list(tactics.Tactic.unique()))])
    for week in subset.Week.unique():
        for media in subset.Media.unique():
            data.loc[(data['year'] == year) & (data['week'] == week),
                             [media]] = subset.loc[(subset['Week'] == week) & 
                                                   (subset['Media'] == media),['Spending']].iloc[0,0]
        for types in tactics.Tactic.unique():
            data.loc[(data['year'] == year) & (data['week'] == week),
                             [types]] = subset.loc[(subset['Week'] == week) & 
                                                   (subset['Tactic'] == types),['Spending']].iloc[0,0]
    data.to_csv(filename)
    
#
addMedia(volume_byproduct, spending, 2020, 'DATA/Clean/2020VolumeByProduct.csv')
addMedia(volume_byproduct, spending, 2019, 'DATA/Clean/2019VolumeByProduct.csv')
addMedia(volume_byproduct, spending, 2018, 'DATA/Clean/2018VolumeByProduct.csv')
#
addMedia(dollar_byproduct, spending, 2020, 'DATA/Clean/2020DollarByProduct.csv')
addMedia(dollar_byproduct, spending, 2019, 'DATA/Clean/2019DollarByProduct.csv')
addMedia(dollar_byproduct, spending, 2018, 'DATA/Clean/2018DollarByProduct.csv')
#
addMedia(unit_byproduct, spending, 2020, 'DATA/Clean/2020UnitByProduct.csv')
addMedia(unit_byproduct, spending, 2019, 'DATA/Clean/2019UnitByProduct.csv')
addMedia(unit_byproduct, spending, 2018, 'DATA/Clean/2018UnitByProduct.csv')
#
addMedia(price_byproduct, spending, 2020, 'DATA/Clean/2020PriceByProduct.csv')
addMedia(price_byproduct, spending, 2019, 'DATA/Clean/2019PriceByProduct.csv')
addMedia(price_byproduct, spending, 2018, 'DATA/Clean/2018PriceByProduct.csv')
#
addMedia(volume_byweek, spending, 2020, 'DATA/Clean/2020VolumeByWeek.csv')
addMedia(volume_byweek, spending, 2019, 'DATA/Clean/2019VolumeByWeek.csv')
addMedia(volume_byweek, spending, 2018, 'DATA/Clean/2018VolumeByWeek.csv')
#
addMedia(dollar_byweek, spending, 2020, 'DATA/Clean/2020DollarByWeek.csv')
addMedia(dollar_byweek, spending, 2019, 'DATA/Clean/2019DollarByWeek.csv')
addMedia(dollar_byweek, spending, 2018, 'DATA/Clean/2018DollarByWeek.csv')
#
addMedia(unit_byweek, spending, 2020, 'DATA/Clean/2020UnitByWeek.csv')
addMedia(unit_byweek, spending, 2019, 'DATA/Clean/2019UnitByWeek.csv')
addMedia(unit_byweek, spending, 2018, 'DATA/Clean/2018UnitByWeek.csv')
#
addMedia(price_byweek, spending, 2020, 'DATA/Clean/2020PriceByWeek.csv')
addMedia(price_byweek, spending, 2019, 'DATA/Clean/2019PriceByWeek.csv')
addMedia(price_byweek, spending, 2018, 'DATA/Clean/2018PriceByWeek.csv')